In [33]:

%run closest_versus_minimal_tools.ipynb

In [34]:
import itertools


def descent(jsonfilename):
    cases = from_json(jsonfilename)
    reached = []
    pathways = []
    for case in cases:
        B, lc_cube, lc_LLL = case["B"], case["lincomb_cube"], [int(i) for i in case["lincomb_LLL"]]
        r, pathway = descent_single(B, lc_cube, lc_LLL)
        reached.append(r)
        pathways.append(pathway)
        print(pathway[1:])
        plot_pathway_and_nearest_norms(pathway,r, case["lincomb_cube"], case["lincomb_LLL"], case["B"], neighborhood=3 )
        print("----------------------------")
    return reached, pathways


def descent_single(B, lc_cube, lc_LLL):
    """
    The last appended point is always the LLL lc. 
    """
    pathway = []
    index = get_index_to_remove(lc_cube, lc_LLL)
    # Append the first, real-valued point of the path. This is used later for plotting.
    pathway.append(lc_cube)

    # Find the point of the small cube with smallest norm and set it as a center
    print("cube_points(lc_cube): ", cube_points(lc_cube))
    print("evaluate_norms_at_cube: ", evaluate_norms_at_cube(cube_points(lc_cube), B))
    min_norm, min_lc = evaluate_norms_at_cube(cube_points(lc_cube), B)[0]
    center = min_lc

    while True :
        pathway.append(center)

        print("CURRENT LOOP: ")
        print("min_norm",min_norm)
        print("center", center)

        big_cube = big_cube_points(min_lc, index)
        print("big_cube:", big_cube)


        bigCubeMinimum_norm, bigCubeMinimum_lc = evaluate_norms_at_cube(big_cube, B)[0]
        print("bigCubeMinimum_norm, bigCubeMinimum_lc:", bigCubeMinimum_norm, bigCubeMinimum_lc)

        # If the new minimum is bigger than the old
        if min_norm < bigCubeMinimum_norm:
            reached = False
            print("the new minimum is bigger than the old")
            break

        # If I reached the lcLLL within the, end with success status
        if bigCubeMinimum_lc == lc_LLL:
            pathway.append(bigCubeMinimum_lc)
            reached = True
            print("I reached the lcLLL within the, end with success status")
            break

        # # If I got stuck in a local minimum, end with fail status
        # elif center == bigCubeMinimum_lc:
        #     reached = False
        #     print("I got stuck in a local minimum, end with fail status")
        #     break 

        # Else, continue with the descent
        min_norm, center = bigCubeMinimum_norm, bigCubeMinimum_lc
    pathway.append(lc_LLL)
    return reached, pathway




def big_cube_points(int_combination, index):
    points = []
    for i in range(len(int_combination)):
        if i == index:
            continue
        dupe = int_combination[:]
        dupe[i] += 1
        points.append(copy(dupe))
        for j in range(len(int_combination)):
            if j == index or j== i:
                continue
            dupe_inner = dupe[:]
            dupe_inner[j] += 1
            points.append(copy(dupe_inner))
            dupe_inner[j] += -2
            points.append(copy(dupe_inner))
        dupe[i] -= 2
        points.append(copy(dupe))        
    points = [list(map(lambda x : int(x), point)) for point in points]
    return points




def vector_difference(u, v):
    difference = 0
    for a, b in zip(u, v):
        difference += abs(a - b)
    return difference


# This is just experimental function to see if more flexibility with the bigger cube gives better results. (I didnt)

def BIG_big_cube_points(point):
    neighbors = []
    n = len(point)

    # Generate all possible combinations of -1, 0, and 1 for each dimension
    directions = list(itertools.product([-1, 0, 1], repeat=n))

    # Remove the combination with all zeros, as this corresponds to the input point itself
    directions.remove(tuple([0] * n))

    # Generate neighboring points by adding each direction to the input point
    for direction in directions:
        neighbor = [point[i] + direction[i] for i in range(n)]
        neighbors.append(neighbor)

    return neighbors



What if I start the ascent in the closest point? It doesnt make much sense, as I am not able to compute the closest point without knowledge of the LLL linear combination, but im just curious rn.

In [35]:
def descent_from_closest(jsonfilename):
    cases = from_json(jsonfilename)
    reached = 0
    for case in cases:
        B, lc_cube, lc_LLL = case["B"], case["lincomb_cube"], case["lincomb_LLL"]
        minimal =  closest_point_in_cube(lc_cube, lc_LLL)
        center = minimal
        central_norm = (vector(minimal)*matrix(B)).norm().n()
        while True :
            big_cube = big_cube_points(minimal)
            bigCubeMinimum = evaluate_norms_at_cube(big_cube, B)[0][1]
            if center == bigCubeMinimum:
                break
            if bigCubeMinimum == lc_LLL:
                reached += 1
            center = bigCubeMinimum
    return reached, len(cases) - reached


In [36]:
# This part of code is meant to generate the precise points the algorithm took during the descent. 


import matplotlib.pyplot as plt

def different_components(pathway):
    diff_indices = set(range(len(pathway[0])))
    
    for i in range(len(pathway[0])):
        unique_values = {point[i] for point in pathway}
        if len(unique_values) == 1:
            diff_indices.remove(i)
            break

    return list(diff_indices)

def plot_pathway(pathway, result):
    '''
    Plots the pathway of the descent algorithm in following manner:

    red dot - lcCube
    blue small dots - pathway of the descent
    blue big dot - end of the descent, if not seen the descent ended successfully
    orange big dot - lcLLL
    '''
    diff_indices = different_components(pathway)

    x = [point[diff_indices[0]] for point in pathway]
    y = [point[diff_indices[1]] for point in pathway]
    
    # Create the plot
    plt.scatter(x[-1], y[-1], color='orange',s=70)  # Highlight the last point lcLLL in green
    plt.scatter(x[1:-1], y[1:-1], color='blue',s=10)
    plt.scatter(x[0], y[0], color='red')  # Highlight the first point lcCUBE in red

    # Add description next to the red point
    description = f"({x[0]:.2f}, {y[0]:.2f})"
    plt.text(x[0] + 0.1, y[0] - 0.1, description)

    # Add descriptions to the blue points
    for i in range(1, len(x) - 1):
        description = f"({x[i]:.2f}, {y[i]:.2f})"
        plt.text(x[i] + 0.1, y[i] - 0.1, description)

    # Add gridlines corresponding to integers
    plt.grid(True, which='both')

    # Set axis ticks to integers only
    plt.xticks(range(int(min(x)) - 1, int(max(x)) + 2))
    plt.yticks(range(int(min(y)) - 1, int(max(y)) + 2))

    # Show the plot
    print(result)
    plt.show()

In [37]:
def plot_pathway_and_nearest_norms(pathway, result,lcCube, lcLLL, B, neighborhood ):
    plt.figure(figsize=(2*neighborhood,2*neighborhood )) 

    diff_indices = different_components(pathway)

    x = [point[diff_indices[0]] for point in pathway]
    y = [point[diff_indices[1]] for point in pathway]
    
    # Create the plot
    plt.scatter(x[-1], y[-1], color='green',s=90)  # Highlight the last point lcLLL in green
    plt.scatter(x[1:-1], y[1:-1], color='orange',s=70) 
    plt.scatter(x[0], y[0], color='red', s=60)  # Highlight the first point lcCUBE in red

    # Add description next to the red point
    description = f"({x[0]:.2f}, {y[0]:.2f})"
    plt.text(x[0] + 0.1, y[0] - 0.1, description)

    # Add descriptions to the blue points
    for i in range(1, len(x) - 1):
        description = f"({x[i]:.2f}, {y[i]:.2f})"
        plt.text(x[i] + 0.1, y[i] - 0.1, description)

    # Add gridlines corresponding to integers
    plt.grid(True, which='both')

    # Set axis ticks to integers only
    plt.xticks(range(int(min(x)) - 1, int(max(x)) + 2))
    plt.yticks(range(int(min(y)) - 1, int(max(y)) + 2))

    index = get_index_to_remove(lcCube, lcLLL)
    nearby_integer_lcs = generate_nearby_integer_points(lcCube, index, neighborhood)

    nearby_removed, lcCubeRemoved , lcLLLRemoved= remove_coordinate(index, nearby_integer_lcs, lcCube, lcLLL)

    x_coords = [point[0] for point in nearby_removed]
    y_coords= [point[1] for point in nearby_removed]
    norms = [(vector(point)*matrix(B)).norm().n(digits=3) for point in nearby_integer_lcs]

    norm_min = min(norms)
    norm_max = max(norms)
    colors = [(norm - norm_min) / (norm_max - norm_min) for norm in norms]


    plt.scatter(x_coords, y_coords, c=colors, cmap='Blues', edgecolors='face', s=30)

    # plt.scatter(x_coords, y_coords, color='blue',s=30)
    # plt.scatter(lcCubeRemoved[0], lcCubeRemoved[1], color="red", s=40)
    # plt.scatter(lcLLLRemoved[0], lcLLLRemoved[1], color="green", s=40)

    # Add norm as a description to each point with coords [x, y]
    for x, y, norm in zip(x_coords, y_coords, norms):
        plt.annotate(
            f"{norm}",
            xy=(x, y),
            xytext=(x + 0.1, y + 0.1),
            fontsize=10,
        )

    # Annotate the lcCube's norm
    plt.annotate(
            (vector(lcCube)*matrix(B)).norm().n(digits=3),
            xy=(lcCubeRemoved[0], lcCubeRemoved[1]),
            xytext=(lcCubeRemoved[0] + 0.1, lcCubeRemoved[1] + 0.1),
            fontsize=10,
            color="red",
        )    
    
    # Add gridlines corresponding to integers
    plt.grid(True, which='both')

    # Set axis ticks to integers only
    plt.xticks(range(int(min(x_coords)) - 1, int(max(x_coords)) + 2))
    plt.yticks(range(int(min(y_coords)) - 1, int(max(y_coords)) + 2))
    
    tmp = B[:]
    tmp.pop(index)

    # Show the plot
    print(result)
    plt.show()

def plot_nearest_norms(lcCube, lcLLL, B, neighborhood):
    '''
    Plot the nearest norms of lcCube and lcLLL, as well as nearby integer points.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
        B (list of lists): The basis matrix.
        neighborhood (int): The maximum distance to generate nearby integer points.
    '''
    
    # Show the plot
    print("Deleted index: ", index)
    print("x-axis: vector {} with norm {}".format(tmp[0], vector(tmp[0]).norm().n(digits=4)))
    print("y-axis: vector {} with norm {}".format(tmp[1], vector(tmp[1]).norm().n(digits=4)))
    plt.show()

In [38]:

def get_index_to_remove(lcCube, lcLLL):
    '''
    Find the index of the identical coordinate shared by lcCube and lcLLL.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        lcLLL (list): The lcLLL point represented as a list of coordinates.
    '''
    if len(lcCube) != len(lcLLL):
        raise ValueError("Both input lists must have the same length")

    for index, (elem_lcCube, elem_lcLLL) in enumerate(zip(lcCube, lcLLL)):
        if elem_lcCube == elem_lcLLL:
            return index
    return ValueError("The points do not share identical coordinate.")


def generate_nearby_integer_points(lcCube, exception_index, dist):
    '''
    Generate nearby integer points of lcCube within a given distance, excluding a specific index.

    INPUTS:
        lcCube (list): The lcCube point represented as a list of coordinates.
        exception_index (int): The index of the coordinate to exclude from the calculation.
        dist (int): The maximum distance to generate nearby integer points.
    '''
    n = len(lcCube)
    nearby_points = []
    
    # Calculate the range of possible integer coordinates for each dimension, except at index ind
    ranges = [range(int(round(coord) - dist + 1), int(round(coord) + dist)) if i != exception_index else [lcCube[exception_index]] for i, coord in enumerate(lcCube)]

    # Generate all possible combinations of integer coordinates within the ranges
    coordinate_combinations = list(itertools.product(*ranges))

    for point in coordinate_combinations:
        total_diff = sum(abs(lcCube[i] - point[i]) for i in range(n))
        if total_diff < dist:
            nearby_points.append(list(point))
    
    return nearby_points


def remove_coordinate(index, points, lcCube, lcLLL):
    '''
    Remove a coordinate from a list of points, lcCube, and lcLLL at the specified index.
    '''
    filtered_points = [remove_coordinate_single(index, point) for point in points]
    lcCubeRemoved = remove_coordinate_single(index, lcCube)
    lcLLLRemoved = remove_coordinate_single(index, lcLLL)
    return filtered_points, lcCubeRemoved, lcLLLRemoved


def remove_coordinate_single(index, point):
    '''
    Remove a coordinate from a single point at the specified index.
    INPUTS:
        index (int): The index of the coordinate to be removed.
        point (list): The point represented as a list of coordinates.
    '''
    point_removed = point[:]
    point_removed.pop(index)
    return point_removed